# Imports


In [ ]:
import numpy as np 
import pandas as pd
import datetime

# interactive plotting with holoviews/bokeh.
# Holoviews uses datashader as backend
import holoviews as hv
from holoviews.operation.datashader import datashade
hv.extension('bokeh')

# **Key functionality**

In [ ]:
def readMaster(master):
    """
    Function to read the master data file (cleaned and collated data for all devices)
   
    args:
    master: string name of the master file with file extension, .csv
    
    return:
    df: pandas dataframe containing the loaded data
    """
    df = pd.read_csv(master)
    df.drop(columns=['Unnamed: 0'], inplace=True)
    return df

In [ ]:
def plotPMOverlay(curr_dev):
    """
    Function to plot an overlay of different PM measurement time series (1.0, 2.5, 10.0) for a particular device
    
    args:
    curr_dev: `device` field of the device in question 
    
    return:
    scatter: overlay plot holoviews object. To display it, use: `hv.output(scatter)`
    """
    x = pd.to_datetime(curr_dev['when_captured']).values
    line1 = hv.Scatter((x,curr_dev['pms_pm10_0']), label='PM10.0').opts(width=800, xlabel = 'date', ylabel = 'PMx')
    line2 = hv.Scatter((x,curr_dev['pms_pm02_5']), label='PM2.5').opts(width=800)
    line3 = hv.Scatter((x,curr_dev['pms_pm01_0']), label='PM1.0').opts(width=800)
    scatter = line1*line2*line3
    scatter.opts(legend_position='top_right')
    return scatter

In [ ]:
def plotRadiationOverlay(curr_dev):
    """
    Function to plot an overlay of different radiation measurement time series for a particular device
    
    args:
    curr_dev: `device` field of the device in question 
    
    return:
    scatter: overlay plot holoviews object. To display it, use: `hv.output(scatter)`
    """
    x = pd.to_datetime(curr_dev['when_captured']).values
    line1 = hv.Scatter((x,curr_dev['lnd_7318u']), label='lnd_7318u')\
    .opts(width=800, xlabel = 'date', ylabel = 'Counts per minute (CPM)')
    line2 = hv.Scatter((x,curr_dev['lnd_7318c']), label='lnd_7318c').opts(width=800)
    scatter = line1*line2
    scatter.opts(legend_position='top_right')
    return scatter

In [ ]:
def negativeField(field, curr_dev):
    """
    Function to filter anomlous records based on negative (therefore meaningless) values of the `field`
    
    args:
    field: string name of the field of interest for anomaly detection 
    curr_dev: `device` field of the device in question 
    
    return:
    faults: pandas dataframe with containing anaomalous records: 4 fields are in the list
            ['anomaly_type','device','normalized_severity_score','when_captured']
    """
    faults = curr_dev[curr_dev[field] < 0][['when_captured', 'device']]
    faults['anomaly_type'] = np.repeat(field + ' < 0', faults.shape[0])
    faults['normalized_severity_score'] = 1
    return faults

In [ ]:
def rollingMeanDev(field, curr_dev, window, min_period, numStd):
    """
    Function to filter anomlous records based on `numStd` number of deviations away from rolling mean
    
    args:
    field: string name of the field of interest for anomaly detection 
    curr_dev: `device` field of the device in question 
    window: moving window size for the rolling mean and stddev
    min_period: Minimum number of observations in window required to have a value (otherwise result is NA)
    numStd: tolerance in number of standard deviations away from mean for record to be anomalous
    
    return:
    pandas dataframe with containing anaomalous records: 4 fields are in the list
            ['anomaly_type','device','normalized_severity_score','when_captured']
    overlay: holoviews plot object with running mean, +/- numStd lines, and data
    """
    rollingMean = curr_dev[field].rolling(window, min_periods=min_period).mean()
    rollingStdev = curr_dev[field].rolling(window, min_periods=min_period).std()
    upper = rollingMean + (rollingStdev * numStd)
    lower = rollingMean - (rollingStdev * numStd)
    
    # visualize
    x = pd.to_datetime(curr_dev['when_captured']).values
    line1 = hv.Scatter((x,curr_dev[field]), label='data').opts(width=700, ylabel = field)
    line1Mean = hv.Curve((x, rollingMean), label='mean').opts(width=700, color='red', xlabel = 'date')
    line1Upper = hv.Curve((x, upper), label='mean+ 3.stdev').opts(width=700, color='blue')
    line1Lower = hv.Curve((x, lower), label='mean- 3.stdev').opts(width=700, color='blue')
    
    overlay = line1Upper * line1Mean * line1 * line1Lower
    overlay # there exists a save command too

    # return the list of anomalies : records where deviation is >= num_std away from mean
    temp = curr_dev.copy()
    temp['rollingMean'] = rollingMean
    temp['rollingStdev'] = rollingStdev
    temp['normalized_severity_score'] = (temp[field]-temp['rollingMean'])/(numStd*temp['rollingStdev'])
    temp = temp[abs(temp['normalized_severity_score']) >= 1]
    
    # instead of string for anomaly type, we can also make a key:value dictionary 
    # and use that as meta-data for reporting
    temp['anomaly_type'] = np.repeat(field + ' ' + str(numStd) + ' or more away from mean', temp.shape[0])
    return temp[['anomaly_type', 'device', 'normalized_severity_score', 'when_captured']], overlay

# **Example Run**

### Read the master csv file with data for all Solarcast devices since inception
There are about 36 different Solarcast deployments.

Keep the master data file in the same folder as this file

In [ ]:
master = 'Solarcast-01_Master_Cleaned_Sorted.csv'
df = readMaster(master)

### Group by `device` field
Note that this `device` field is the unique device identifier (36 such). A single device can have different `device_urn` or `device_sn`

In [ ]:
gf = df.groupby('device') # remember the key is "device" not "device_sn or device_urn"
devices = np.unique(df.device)

In [ ]:
curr_dev  = gf.get_group(devices[0])

### Plotting AQ datapoints

In [ ]:
plotPMOverlay(curr_dev)

### Plot the radiation data

In [ ]:
plotRadiationOverlay(curr_dev)

## **Automated (aka unsupervised) anomaly detection**
Flag errors for each of the following situtions -- idea is to draw attention of Safecast personnel

*To Do:* **Need to expand it to more than 1 device (involves a loop over all the devices created by `grouping`)**

We also want to tag the resulting `faulty` time stamps indicating which (one or more *To Do:* more?) among the below anomalies kicked in:

0. If PM or radiation counts (CPM) are negative -- those are anomalies for sure
1. 3 "rolling stddev" deviation away from mean (outside the 95% confidence interval)
2. *To Do:* ? Poor rolling correlation between two time series at certain instances ?
3. *To Do:* ? Add functionality for anomalies when there are no readings for a device for more than 2 days ?
4. *To Do:* more from discussion on 08/13/2020?

### Dataframes for storing reportable anomalies
Create `anomalyf_air` and `anomalyf_rad` dataframes to contain list of all anomalies found for a particular device (separate dfs for air and radiation data

In [ ]:
cols = ['anomaly_type','device','normalized_severity_score','when_captured']
anomalyf_air = pd.DataFrame(columns=cols)
anomalyf_rad = pd.DataFrame(columns=cols)

### Anomalies: Negative PM or CPM 

In [ ]:
for field in ['pms_pm10_0', 'pms_pm02_5', 'pms_pm01_0']:
    anomalyf_air = anomalyf_air.append(negativeField(field, curr_dev), ignore_index=True, sort=True)
    
# for radiation
for field in ['lnd_7318u', 'lnd_7318c']:
    anomalyf_rad = anomalyf_rad.append(negativeField(field, curr_dev), ignore_index=True, sort=True)

### Anomalies: 3 std away from mean

3 std-dev deviation away from rolling mean:

Tag as anomaly anything that is, say, 3 standard deviations away from the moving average.
Notice that it's hard to tell what is an anomaly without making an assumption of what is normal i.e. what is the data generating process.

*To Do:* Examine the effect of the `window` and `min_period` variables

In [ ]:
# parameters -- can be a different set for radiation and AQ measurements
window = 300
min_period = 100
numStd = 3

#store all plot objects in a list
plots= []

for field in ['pms_pm10_0', 'pms_pm02_5', 'pms_pm01_0']:
    anom, plot = rollingMeanDev(field, curr_dev, window, min_period, numStd)
    anomalyf_air = anomalyf_air.append(anom, ignore_index=True, sort=True)
    plots.append(plot)

# for radiation
for field in ['lnd_7318u', 'lnd_7318c']:
    anom, plot = rollingMeanDev(field, curr_dev, window, min_period, numStd)
    anomalyf_rad = anomalyf_rad.append(anom, ignore_index=True, sort=True)
    plots.append(plot)

Note that the lower bound in plots below for the AQ msmts is meaningless. Just there for consistencyin code

In [ ]:
for plot in plots:
    hv.output(plot)

In [ ]:
anomalyf_air

In [ ]:
print("percent anomalous data = ", anomalyf_air.shape[0]/curr_dev.shape[0]*100) 
# check for uniqueness of when_captured by using group by

In [ ]:
anomalyf_rad

In [ ]:
print("percent anomalous data = ", anomalyf_rad.shape[0]/curr_dev.shape[0]*100) 
